# Confusion matrixes for Spacy Model Testing

In [169]:
def confusion_matrix(docs, model_name, model_path, docs_path, entity_list):
    '''
    docs: list of names of test set docs from training spacy nlp model
    model_name: name of spacy nlp model to test
    model_path: pathname to model
    docs_path: pathname to docs
    '''
    #import spacy and load model
    
    import spacy 
    import os
    import pandas as pd
    os.chdir(model_path)
    nlp_ner = spacy.load(model_name)
    
    #change directory to doc path
    
    os.chdir(docs_path)
    
    #define ann to dict helper function
    
    def ann_to_dict(nct_id): 
        '''
        nct_id: file name before .ann or .txt
        file_path: pathname to where .ann and .txt files are stored
        return: formatted dictionary
        '''

        # read in files
        with open(nct_id+'.ann') as f1:
            ann=f1.read()
            f1.close()
        with open(nct_id+'.txt') as f2:
            txt=f2.read()
            f2.close()

        # get list of spans
        lines=ann.split('\n')
        ents=[]
        for i in range(len(lines)): 
            line=lines[i].split('\t') 
            if 'T' in line[0]:
                try: 
                    start=txt.index(line[2])
                    end= start+len(line[2])
                    label=line[1].split(' ')[0]
                    if (label in entity_list):
                        ents.append((start, end, label))
                except:
                    None

        content={'entities': list(set(ents)), 'text': txt}

        return content
    
    # intialize dictionaries to track counts
    tp={'all':0}
    fp={'all':0}
    fn={'all':0}
    n_pred={'all':0}
    n_true={'all':0}
    
    for entity in entity_list:
        tp[entity]=0
        fp[entity]=0
        fn[entity]=0
        n_pred[entity]=0
        n_true[entity]=0

    # parse through docs and add to confusion matrix variables

    for x in docs: 
        
        # get true ents and predicted ents
        trial=ann_to_dict(x)
        true_ents=trial['entities']
        doc = nlp_ner(trial['text'])
        pred_ents = [(e.start_char, e.end_char, e.label_) for e in doc.ents]
        
        # add true ents to n counts
        for true in true_ents:
            n_true[true[2]]+=1
            n_true['all']+=1
        
        # interate through predicted ents and classify
        found_list=[]
        for pred in pred_ents:
            # add predicted ents to n counts
            n_pred[pred[2]]+=1
            n_pred['all']+=1
            
            match=False 
            for true in true_ents: #check if match to a true ent
                if pred[2]==true[2]: #if labels match... 
                    #if spans overlap...
                    if ((pred[0]<=true[0]<=pred[1]) or (pred[0]<=true[1]<=pred[1]) or (true[0]<=pred[0]<=true[1]) or (true[0]<=pred[1]<=true[1])):
                    #add to true positives
                        tp['all']+=1
                        tp[pred[2]]+=1
                        found_list.append(true)
                        match=True
                if match==True: #if match found, stop iterating
                    break

            if match==False: #if match never found, add to false positives
                fp['all']+=1
                fp[pred[2]]+=1
                
        # count true ents that were not matched as false positives
        for x in [x for x in true_ents if x not in found_list]:
            fn['all']+=1
            fn[true[2]]+=1
    
    # calculate precision, recall, F1:
    
    #helper functions
    def p(ent):
        try:
            return tp[ent]/(tp[ent]+fp[ent])
        except:
            return None
    def r(ent):
        try:
            return tp[ent]/(tp[ent]+fn[ent])
        except:
            return None
    def f1(ent):
        try:
            return (2*p(ent)*r(ent))/(p(ent)+r(ent))
        except:
            return None
    
    # add to results dict
    results={'entity':['p', 'r', 'f', 'n_true', 'n_pred'], 'all':[p('all'), r('all'), f1('all'), n_true['all'], n_pred['all']]}
    for entity in entity_list:
        results[entity]=[p(entity), r(entity), f1(entity), n_true[entity], n_pred[entity]]
        
    
    #print results:
    print('True Positives:')
    print(tp)
    print('False Positives:')
    print(fp)
    print('False Negativies:')
    print(fn)
    print('Overall Scores:')
    return pd.DataFrame(results)

In [170]:
model_name='model-best'
model_path='/Users/meldrumapple/Desktop/Capstone/mod_chia_noscope'
corpus_path='/Users/meldrumapple/Desktop/Capstone/chia_noscope_corpus'

In [171]:
label_list=['Person', 'Condition', 'Drug', 'Observation', 'Measurement', 'Procedure', 'Device', 'Visit', 'Negation', 'Qualifier', 'Temporal', 'Value', 'Multiplier', 'Reference_point', 'Mood','Post-eligibility', 'Pregnancy_considerations', 'Informed_consent']

In [172]:
test_docs=['NCT01650792_exc', 'NCT03387059_exc', 'NCT02707809_exc', 'NCT03539718_inc', 'NCT02245256_exc', 'NCT01312012_inc', 'NCT02937779_exc', 'NCT02678377_exc', 'NCT02489045_inc', 'NCT02042287_exc', 'NCT02570347_exc', 'NCT03228017_inc', 'NCT02167022_exc', 'NCT02222272_inc', 'NCT01929434_inc', 'NCT02015494_inc', 'NCT02698969_exc', 'NCT00061308_exc', 'NCT02764476_inc', 'NCT02528604_inc', 'NCT00970866_inc', 'NCT02652572_exc', 'NCT02787863_inc', 'NCT00500500_exc', 'NCT00586898_exc', 'NCT02664558_exc', 'NCT03329456_inc', 'NCT00455663_inc', 'NCT03297944_inc', 'NCT01184638_inc', 'NCT02701777_inc', 'NCT03134196_exc', 'NCT02953873_exc', 'NCT02019628_inc', 'NCT02121145_exc', 'NCT00994786_inc', 'NCT02339844_inc', 'NCT03278548_exc', 'NCT02704234_exc', 'NCT02704754_inc', 'NCT02528136_inc', 'NCT00461136_exc', 'NCT02406495_inc', 'NCT03236246_exc', 'NCT03337581_inc', 'NCT00812344_exc', 'NCT03045562_exc', 'NCT03156855_exc', 'NCT02745704_exc', 'NCT02364648_inc', 'NCT02650024_inc', 'NCT02883400_exc', 'NCT02951832_exc', 'NCT02900443_inc', 'NCT01314898_exc', 'NCT02231892_exc', 'NCT03088904_inc', 'NCT01604187_exc', 'NCT01816997_inc', 'NCT02393287_inc', 'NCT01491763_exc', 'NCT02612181_inc', 'NCT02958072_exc', 'NCT02499185_exc', 'NCT03119766_inc', 'NCT03264911_exc', 'NCT01978028_inc', 'NCT02924090_inc', 'NCT02992028_inc', 'NCT03058835_exc', 'NCT02330757_inc', 'NCT03305575_exc', 'NCT01205334_exc', 'NCT02638935_inc', 'NCT00599924_exc', 'NCT03063866_inc', 'NCT02426944_exc', 'NCT02630628_exc', 'NCT02612181_exc', 'NCT00050349_exc', 'NCT02202369_inc', 'NCT02894645_exc', 'NCT03355157_exc', 'NCT02348918_exc', 'NCT02613039_exc', 'NCT01602081_inc', 'NCT03420638_inc', 'NCT02687178_inc', 'NCT02652637_inc', 'NCT02785549_inc', 'NCT02637076_inc', 'NCT02990403_inc', 'NCT02432404_exc', 'NCT01743755_inc', 'NCT03532620_inc', 'NCT02467686_exc', 'NCT00806273_exc', 'NCT02637453_inc', 'NCT03299517_exc', 'NCT03315975_inc', 'NCT00480129_inc', 'NCT02715466_exc', 'NCT03255044_exc', 'NCT02821819_exc', 'NCT03472495_exc', 'NCT03103204_inc', 'NCT03217409_exc', 'NCT01728194_inc', 'NCT03019562_exc', 'NCT02765217_exc', 'NCT02256956_inc', 'NCT03177837_inc', 'NCT00749112_inc', 'NCT03034733_inc', 'NCT02416869_exc', 'NCT03352869_exc', 'NCT02283905_inc', 'NCT02937779_inc', 'NCT02385448_exc', 'NCT03122119_inc', 'NCT02499185_inc', 'NCT01715714_exc', 'NCT03084588_exc', 'NCT01117181_exc', 'NCT02334631_exc', 'NCT03099863_inc', 'NCT03067740_inc', 'NCT00344318_inc', 'NCT00959569_exc', 'NCT03213834_exc', 'NCT02823808_exc', 'NCT02445339_inc', 'NCT01631058_exc', 'NCT02396732_exc', 'NCT01665417_exc', 'NCT03125057_inc', 'NCT02548013_exc', 'NCT02477280_exc', 'NCT03335436_exc', 'NCT01614041_inc', 'NCT01696617_inc', 'NCT02765217_inc', 'NCT02787070_inc', 'NCT02838810_exc', 'NCT02269137_exc', 'NCT03555526_inc', 'NCT02227992_exc', 'NCT02019160_exc', 'NCT02851888_exc', 'NCT03209011_inc', 'NCT02394158_exc', 'NCT03388840_inc', 'NCT02399033_inc', 'NCT02618057_inc', 'NCT02056301_inc', 'NCT02375295_exc', 'NCT03350815_exc', 'NCT03476850_exc', 'NCT02667730_exc', 'NCT03288428_inc', 'NCT03539718_exc', 'NCT02357654_exc', 'NCT02749617_inc', 'NCT03511521_inc', 'NCT03228238_inc', 'NCT02973035_exc', 'NCT00236340_inc', 'NCT00425789_inc', 'NCT02473809_exc', 'NCT03129555_inc', 'NCT03297125_exc', 'NCT02789111_inc', 'NCT02609425_inc', 'NCT02701777_exc', 'NCT01943812_inc', 'NCT03335436_inc', 'NCT03320057_exc', 'NCT02464865_exc', 'NCT03363295_exc', 'NCT03226080_exc', 'NCT00965900_exc', 'NCT02318446_exc', 'NCT03382106_inc', 'NCT03004209_exc', 'NCT02106598_exc', 'NCT02301962_inc', 'NCT03382106_exc', 'NCT02957305_exc', 'NCT01650792_inc', 'NCT01866800_exc', 'NCT03088280_inc', 'NCT02983214_inc', 'NCT02150590_exc', 'NCT02607319_exc', 'NCT02529475_inc', 'NCT02106624_exc', 'NCT03125057_exc', 'NCT03372304_exc', 'NCT02974660_exc', 'NCT02247128_exc']

In [173]:
results=confusion_matrix(test_docs, model_name, model_path, corpus_path, label_list)

/Users/meldrumapple/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


True Positives:
{'all': 2707, 'Person': 118, 'Condition': 872, 'Drug': 231, 'Observation': 77, 'Measurement': 225, 'Procedure': 221, 'Device': 31, 'Visit': 13, 'Negation': 48, 'Qualifier': 279, 'Temporal': 207, 'Value': 285, 'Multiplier': 50, 'Reference_point': 0, 'Mood': 38, 'Post-eligibility': 0, 'Pregnancy_considerations': 0, 'Informed_consent': 12}
False Positives:
{'all': 145, 'Person': 13, 'Condition': 14, 'Drug': 8, 'Observation': 15, 'Measurement': 8, 'Procedure': 21, 'Device': 2, 'Visit': 0, 'Negation': 3, 'Qualifier': 20, 'Temporal': 19, 'Value': 16, 'Multiplier': 5, 'Reference_point': 0, 'Mood': 1, 'Post-eligibility': 0, 'Pregnancy_considerations': 0, 'Informed_consent': 0}
False Negativies:
{'all': 526, 'Person': 7, 'Condition': 152, 'Drug': 56, 'Observation': 32, 'Measurement': 24, 'Procedure': 86, 'Device': 11, 'Visit': 0, 'Negation': 1, 'Qualifier': 27, 'Temporal': 35, 'Value': 89, 'Multiplier': 3, 'Reference_point': 0, 'Mood': 0, 'Post-eligibility': 0, 'Pregnancy_consid

In [179]:
results=results.transpose()

In [180]:
results.to_csv('/Users/meldrumapple/Desktop/Capstone/mod_chia_noscope/results.csv')

In [181]:
model_name='model-best'
model_path='/Users/meldrumapple/Desktop/Capstone/mod_lct'
corpus_path='/Users/meldrumapple/Desktop/Capstone/lct_corpus'

In [182]:
label_list=['Allergy','Condition','Contraindication','Drug','Encounter','Indication','Immunization','Observation','Procedure','Age','Birth','Death','Ethnicity','Family-member','Language','Life-Stage-And-Gender','Exception','Negation','Acuteness','Assertion','Modifier','Polarity','Risk','Severity','Stability','Eq-Comparison','Eq-Temporal-Period','Eq-Temporal-Recency','Eq-Temporal-Unit','Eq-Unit']

In [183]:
test_docs=['NCT03927755', 'NCT03868852', 'NCT03860428', 'NCT03865966', 'NCT03864822', 'NCT03929731', 'NCT03927092', 'NCT03922581', 'NCT03862014', 'NCT03866512', 'NCT03867344', 'NCT03862482', 'NCT03925987', 'NCT03868709', 'NCT03865823', 'NCT03866850', 'NCT03922386', 'NCT03925649', 'NCT03929432', 'NCT03867890', 'NCT03863678', 'NCT03929536', 'NCT03864354', 'NCT03866733', 'NCT03865628', 'NCT03927287', 'NCT03860870', 'NCT03861650', 'NCT03921398', 'NCT03921190', 'NCT03924375', 'NCT03864809', 'NCT03861065', 'NCT03862027', 'NCT03934385', 'NCT03920683', 'NCT03928782', 'NCT03864562', 'NCT03865342', 'NCT03924557', 'NCT03868813', 'NCT03929575', 'NCT03861104', 'NCT03920540', 'NCT03927742', 'NCT03866824', 'NCT03925038', 'NCT03861520', 'NCT03860896', 'NCT03927482', 'NCT03865576', 'NCT03928912', 'NCT03868384', 'NCT03927300', 'NCT03929887', 'NCT03861312', 'NCT03927781', 'NCT03921047', 'NCT03928639', 'NCT03923413', 'NCT03868397', 'NCT03865420', 'NCT03868241', 'NCT03866590', 'NCT03925285', 'NCT03926403', 'NCT03930108', 'NCT03861117', 'NCT03867201', 'NCT03926468', 'NCT03925337', 'NCT03925272', 'NCT03924531', 'NCT03863912', 'NCT03862989', 'NCT03865862', 'NCT03927014', 'NCT03868150', 'NCT03929978', 'NCT03869476', 'NCT03926494', 'NCT03868904', 'NCT03863743', 'NCT03868761', 'NCT03924427', 'NCT03928834', 'NCT03860415', 'NCT03929497', 'NCT03866135', 'NCT03925441', 'NCT03924934', 'NCT03926975', 'NCT03869411', 'NCT03920319', 'NCT03921580', 'NCT03928626', 'NCT03925714', 'NCT03920098', 'NCT03928652', 'NCT03926026']

In [184]:
results_lct=confusion_matrix(test_docs, model_name, model_path, corpus_path, label_list)

/Users/meldrumapple/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


True Positives:
{'all': 1597, 'Allergy': 9, 'Condition': 436, 'Contraindication': 9, 'Drug': 59, 'Encounter': 22, 'Indication': 1, 'Immunization': 0, 'Observation': 132, 'Procedure': 147, 'Age': 48, 'Birth': 0, 'Death': 1, 'Ethnicity': 3, 'Family-member': 0, 'Language': 12, 'Life-Stage-And-Gender': 52, 'Exception': 13, 'Negation': 28, 'Acuteness': 17, 'Assertion': 37, 'Modifier': 151, 'Polarity': 18, 'Risk': 6, 'Severity': 24, 'Stability': 18, 'Eq-Comparison': 305, 'Eq-Temporal-Period': 4, 'Eq-Temporal-Recency': 0, 'Eq-Temporal-Unit': 40, 'Eq-Unit': 5}
False Positives:
{'all': 499, 'Allergy': 0, 'Condition': 117, 'Contraindication': 0, 'Drug': 17, 'Encounter': 12, 'Indication': 0, 'Immunization': 0, 'Observation': 57, 'Procedure': 64, 'Age': 6, 'Birth': 1, 'Death': 0, 'Ethnicity': 0, 'Family-member': 0, 'Language': 2, 'Life-Stage-And-Gender': 13, 'Exception': 1, 'Negation': 14, 'Acuteness': 10, 'Assertion': 17, 'Modifier': 82, 'Polarity': 12, 'Risk': 0, 'Severity': 4, 'Stability': 8, '

In [186]:
results_lct=results_lct.transpose()
results_lct.to_csv('/Users/meldrumapple/Desktop/Capstone/mod_lct/results_lct.csv')

In [262]:
#define helper functions that maps lct/chia labels to our labels
fail_list_chia=[] #so we can see which entities aren't being included
def chia_map(label):
    '''
    label (str): the chia label to be mapped to our label system
    '''
    
    map_dict={
    'Person':'Demographic',
    'Condition':'Condition',
    'Drug':'Drug',
    'Observation':'Observation',
    'Measurement':'Observation',
    'Procedure':'Procedure',
    'Device':'Procedure',
    'Visit': 'Encounter',
    'Negation':'Negation',
    'Qualifier':'Qualifier',
    'Temporal':'Temporal',
    'Value':'Value',
    'Multiplier':'Qualifier',
    'Mood':'Indication',
    'Post-eligibility':'Consent',
    'Pregnancy_considerations':'Demographic',
    'Informed_consent':'Consent'
    }
    
    try: 
        new_lab=map_dict[label]
    except:
        new_lab='fail'
        if label not in fail_list_chia:
            fail_list_chia.append(label)
    return new_lab

fail_list_lct=[]
def lct_map(label):
    '''
    label (str): the lct label to be mapped to our label system
    '''
    
    map_dict={
    'Allergy':'Condition',
    'Condition':'Condition',
    'Contraindication':'Indication',
    'Drug':'Drug',
    'Encounter':'Encounter',
    'Indication':'Indication',
    'Immunization':'Drug',
    'Observation':'Observation',
    'Procedure':'Procedure',
    'Age':'Demographic',
    'Birth':'Observation',
    'Death':'Observation',
    'Ethnicity':'Demographic',
    'Family-member':'Observation',
    'Language':'Demographic',
    'Life-Stage-And-Gender':'Demographic',
    'Exception':'Negation',
    'Negation':'Negation',
    'Acuteness':'Qualifier',
    'Assertion':'Indication',
    'Modifier':'Qualifier',
    'Polarity':'Qualifier',
    'Risk':'Qualifier',
    'Severity':'Qualifier',
    'Stability':'Qualifier',
    'Eq-Comparison':'Value',
    'Eq-Temporal-Period':'Temporal',
    'Eq-Temporal-Recency':'Temporal',
    'Eq-Temporal-Unit':'Temporal',
    'Eq-Unit':'Value'
    }
    
    try: 
        new_lab=map_dict[label]
    except:
        new_lab='fail'
        if label not in fail_list_lct:
            fail_list_lct.append(label)
    return new_lab

In [263]:
#Define ANN to Dict function to convert ANN files, works for both corpera
def ann_to_dict(nct_id, chia=False): #took out file path arg
    '''
    nct_id: file name before .ann or .txt
    file_path: pathname to where .ann and .txt files are stored
    return: formatted dictionary
    '''
    
    # change directory
    #os.chdir(file_path)
    
    # read in files
    with open(nct_id+'.ann') as f1:
        ann=f1.read()
        f1.close()
    with open(nct_id+'.txt') as f2:
        txt=f2.read()
        f2.close()
        
        # get list of spans
    lines=ann.split('\n')
    ents=[]
    for i in range(len(lines)): 
        line=lines[i].split('\t') 
        if 'T' in line[0]:
            try: 
                start=txt.index(line[2])
                end= start+len(line[2])
                label=line[1].split(' ')[0]
                if chia==False:
                    label=lct_map(label)
                if chia==True: 
                    label=chia_map(label) 
                if label!='fail':
                    ents.append((start, end, label))
            except:
                None
            
    content={'entities': list(set(ents)), 'text': txt}
    #print(content)
    return content

In [264]:
def confusion_matrix(chia_docs, lct_docs, model_name, model_path, entity_list):
    '''
    docs: list of names of test set docs from training spacy nlp model
    model_name: name of spacy nlp model to test
    model_path: pathname to model
    docs_path: pathname to docs
    '''
    #import spacy and load model
    
    import spacy 
    import os
    import pandas as pd
    os.chdir(model_path)
    nlp_ner = spacy.load(model_name)
    
    #change directory to doc path
    
    #os.chdir(docs_path)
    
    # intialize dictionaries to track counts
    tp={'all':0}
    fp={'all':0}
    fn={'all':0}
    n_pred={'all':0}
    n_true={'all':0}
    
    for entity in entity_list:
        tp[entity]=0
        fp[entity]=0
        fn[entity]=0
        n_pred[entity]=0
        n_true[entity]=0
        
    def eval_doc(x, chia):
        # get true ents and predicted ents
        trial=ann_to_dict(x, chia)
        true_ents=trial['entities']
        doc = nlp_ner(trial['text'])
        pred_ents = [(e.start_char, e.end_char, e.label_) for e in doc.ents]
        
        # add true ents to n counts
        for true in true_ents:
            n_true[true[2]]+=1
            n_true['all']+=1
        
        # interate through predicted ents and classify
        found_list=[]
        for pred in pred_ents:
            # add predicted ents to n counts
            n_pred[pred[2]]+=1
            n_pred['all']+=1
            
            match=False 
            for true in true_ents: #check if match to a true ent
                if pred[2]==true[2]: #if labels match... 
                    #if spans overlap...
                    if ((pred[0]<=true[0]<=pred[1]) or (pred[0]<=true[1]<=pred[1]) or (true[0]<=pred[0]<=true[1]) or (true[0]<=pred[1]<=true[1])):
                    #add to true positives
                        tp['all']+=1
                        tp[pred[2]]+=1
                        found_list.append(true)
                        match=True
                if match==True: #if match found, stop iterating
                    break

            if match==False: #if match never found, add to false positives
                fp['all']+=1
                fp[pred[2]]+=1
                
        # count true ents that were not matched as false positives
        for x in [x for x in true_ents if x not in found_list]:
            fn['all']+=1
            fn[true[2]]+=1

    # parse through docs and add to confusion matrix variables
    os.chdir('/Users/meldrumapple/Desktop/Capstone/chia_noscope_corpus')
    for x in chia_docs: 
        eval_doc(x, True)
    
    os.chdir('/Users/meldrumapple/Desktop/Capstone/lct_corpus')
    for x in lct_docs: 
        eval_doc(x, False)
        
    
    # calculate precision, recall, F1:
    
    #helper functions
    def p(ent):
        try:
            return tp[ent]/(tp[ent]+fp[ent])
        except:
            return None
    def r(ent):
        try:
            return tp[ent]/(tp[ent]+fn[ent])
        except:
            return None
    def f1(ent):
        try:
            return (2*p(ent)*r(ent))/(p(ent)+r(ent))
        except:
            return None
    
    # add to results dict
    results={'entity':['p', 'r', 'f', 'n_true', 'n_pred'], 'all':[p('all'), r('all'), f1('all'), n_true['all'], n_pred['all']]}
    for entity in entity_list:
        results[entity]=[p(entity), r(entity), f1(entity), n_true[entity], n_pred[entity]]
        
    
    #print results:
    print('True Positives:')
    print(tp)
    print('False Positives:')
    print(fp)
    print('False Negativies:')
    print(fn)
    print('Overall Scores:')
    return pd.DataFrame(results)

In [265]:
model_name='model-best'
model_path='/Users/meldrumapple/Desktop/Capstone/mod_lct_chia'

In [271]:
label_list=['Condition', 'Indication', 'Drug', 'Encounter', 'Observation', 'Procedure', 'Demographic', 'Negation','Qualifier', 'Temporal', 'Value', 'Consent']

In [272]:
chia_docs=['NCT03073603_inc', 'NCT02680054_inc', 'NCT02634541_inc', 'NCT02950558_exc', 'NCT02884401_inc', 'NCT03106389_exc', 'NCT02638935_inc', 'NCT00926523_exc', 'NCT03337503_exc', 'NCT00397215_inc', 'NCT03213834_inc', 'NCT02827487_exc', 'NCT02053246_inc', 'NCT03173092_inc', 'NCT02553226_exc', 'NCT03089086_exc', 'NCT03025620_inc', 'NCT03413891_exc', 'NCT01184638_inc', 'NCT03350815_exc', 'NCT02884401_exc', 'NCT03089086_inc', 'NCT03255044_inc', 'NCT01491295_inc', 'NCT01743755_inc', 'NCT03132259_inc', 'NCT02984228_inc', 'NCT02933671_inc', 'NCT02558504_exc', 'NCT03125057_exc', 'NCT02560766_inc', 'NCT03236246_inc', 'NCT02536976_exc', 'NCT02283905_exc', 'NCT02773173_exc', 'NCT02805504_exc', 'NCT02141061_exc', 'NCT01996436_exc', 'NCT02647788_inc', 'NCT03513757_inc', 'NCT02946892_exc', 'NCT02141061_inc', 'NCT02385448_exc', 'NCT02312076_exc', 'NCT03537924_inc', 'NCT03477851_exc', 'NCT03325023_inc', 'NCT00319748_inc', 'NCT03305666_exc', 'NCT03043495_exc', 'NCT03299517_inc', 'NCT03344042_exc', 'NCT02634541_exc', 'NCT02783859_exc', 'NCT02564471_exc', 'NCT00586898_exc', 'NCT02385045_inc', 'NCT02797548_exc', 'NCT00962364_inc', 'NCT02535299_exc', 'NCT00305097_exc', 'NCT00344318_inc', 'NCT03369379_exc', 'NCT02462317_exc', 'NCT02414399_exc', 'NCT02062489_exc', 'NCT03318393_exc', 'NCT02570321_exc', 'NCT02035800_inc', 'NCT03430284_inc', 'NCT02643381_exc', 'NCT02900443_inc', 'NCT03404479_inc', 'NCT00785213_inc', 'NCT02630628_exc', 'NCT02593409_inc', 'NCT02570230_exc', 'NCT03134196_exc', 'NCT03211741_exc', 'NCT03221231_exc', 'NCT02431559_exc', 'NCT02555163_exc', 'NCT02601157_exc', 'NCT03338296_exc', 'NCT02562456_exc', 'NCT03208127_exc', 'NCT02570321_inc', 'NCT02584140_inc', 'NCT02231892_exc', 'NCT00425789_inc', 'NCT02862912_inc', 'NCT03013790_exc', 'NCT02042287_exc', 'NCT02743598_exc', 'NCT03335904_inc', 'NCT02101554_inc', 'NCT00931983_inc', 'NCT01084993_inc', 'NCT02334722_exc', 'NCT00351611_exc', 'NCT02245256_exc', 'NCT01711801_exc', 'NCT02339844_inc', 'NCT01806558_inc', 'NCT03249311_inc', 'NCT03637946_inc', 'NCT01803438_exc', 'NCT02777424_exc', 'NCT03062358_inc', 'NCT02361905_exc', 'NCT01929434_inc', 'NCT02101554_exc', 'NCT01228279_exc', 'NCT01314898_inc', 'NCT03355157_inc', 'NCT01711801_inc', 'NCT03233880_exc', 'NCT02823808_exc', 'NCT02748330_exc', 'NCT03335436_exc', 'NCT02787070_exc', 'NCT02312089_inc', 'NCT02707809_inc', 'NCT03355326_inc', 'NCT03472495_inc', 'NCT02195024_inc', 'NCT02437045_exc', 'NCT03373669_inc', 'NCT03338296_inc', 'NCT02668978_exc', 'NCT03047538_inc', 'NCT02992028_inc', 'NCT02542956_exc', 'NCT02570347_inc', 'NCT02473809_inc', 'NCT01664507_inc', 'NCT03360981_inc', 'NCT02885909_exc', 'NCT03193684_exc', 'NCT03537924_exc', 'NCT03620526_inc', 'NCT02270970_inc', 'NCT03481894_inc', 'NCT01866800_inc', 'NCT02844907_exc', 'NCT02571881_inc', 'NCT00749112_exc', 'NCT02416869_exc', 'NCT02490839_exc', 'NCT02735902_inc', 'NCT01680081_inc', 'NCT01631058_inc', 'NCT02827526_inc', 'NCT02476461_inc', 'NCT03297021_exc', 'NCT02970773_exc', 'NCT03099408_exc', 'NCT03216447_inc', 'NCT02838810_exc', 'NCT03249272_inc', 'NCT03096613_exc', 'NCT03373318_exc', 'NCT02584140_exc', 'NCT01680081_exc', 'NCT02916342_inc', 'NCT02476461_exc', 'NCT00480129_exc', 'NCT02419378_inc', 'NCT00989261_inc', 'NCT03231982_inc', 'NCT01184638_exc', 'NCT02924870_exc', 'NCT02946918_inc', 'NCT01907230_inc', 'NCT02924090_exc', 'NCT03463564_exc', 'NCT02504203_inc', 'NCT02822001_exc', 'NCT02364648_exc', 'NCT03027115_exc', 'NCT02427295_exc', 'NCT02102243_inc', 'NCT03079141_inc', 'NCT02579928_exc', 'NCT01322464_inc', 'NCT02966236_exc', 'NCT03047538_exc', 'NCT02744976_inc', 'NCT02607748_exc', 'NCT03231982_exc', 'NCT03304496_inc', 'NCT03424733_inc', 'NCT01581749_exc', 'NCT01602081_exc', 'NCT02437084_inc', 'NCT03320057_exc', 'NCT03318874_exc', 'NCT02695992_inc', 'NCT03056391_inc', 'NCT00094861_inc']

In [273]:
lct_docs=['NCT03862079', 'NCT03924531', 'NCT03926702', 'NCT03926078', 'NCT03929640', 'NCT03928418', 'NCT03927274', 'NCT03868982', 'NCT03928132', 'NCT03927820', 'NCT03862664', 'NCT03862131', 'NCT03922867', 'NCT03863158', 'NCT03862248', 'NCT03861195', 'NCT03867929', 'NCT03868176', 'NCT03865745', 'NCT03921827', 'NCT03868969', 'NCT03929653', 'NCT03865160', 'NCT03926000', 'NCT03861741', 'NCT03928288', 'NCT03865381', 'NCT03860233', 'NCT03920436', 'NCT03863548', 'NCT03866655', 'NCT03928158', 'NCT03867032', 'NCT03920670', 'NCT03926234', 'NCT03866252', 'NCT03867656', 'NCT03862495', 'NCT03866083', 'NCT03920293', 'NCT03920306', 'NCT03861585', 'NCT03865797', 'NCT03924206', 'NCT03865576', 'NCT03928938', 'NCT03864172', 'NCT03869151', 'NCT03921918', 'NCT03865680', 'NCT03921125', 'NCT03921307', 'NCT03862534', 'NCT03867305', 'NCT03868605', 'NCT03868891', 'NCT03929692', 'NCT03860493', 'NCT03925441', 'NCT03865524', 'NCT03869905', 'NCT03864367', 'NCT03920384', 'NCT03927664', 'NCT03926481', 'NCT03926546', 'NCT03863236', 'NCT03867942', 'NCT03926949', 'NCT03869463', 'NCT03925272', 'NCT03867539', 'NCT03866850', 'NCT03860038', 'NCT03927781', 'NCT03861988', 'NCT03868761', 'NCT03928405', 'NCT03927742', 'NCT03922594', 'NCT03863197', 'NCT03863756', 'NCT03925116', 'NCT03868254', 'NCT03922243', 'NCT03867591', 'NCT03864471', 'NCT03921216', 'NCT03861299', 'NCT03929068', 'NCT03864952', 'NCT03867617', 'NCT03862703', 'NCT03869645', 'NCT03920904', 'NCT03929575', 'NCT03860428', 'NCT03868995', 'NCT03928431', 'NCT03861598']

In [274]:
results_combo=confusion_matrix(chia_docs, lct_docs, model_name, model_path, label_list)

/Users/meldrumapple/opt/anaconda3/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


True Positives:
{'all': 5161, 'Condition': 1681, 'Indication': 90, 'Drug': 409, 'Encounter': 18, 'Observation': 515, 'Procedure': 514, 'Demographic': 291, 'Negation': 109, 'Qualifier': 631, 'Temporal': 204, 'Value': 642, 'Consent': 57}
False Positives:
{'all': 1073, 'Condition': 259, 'Indication': 33, 'Drug': 57, 'Encounter': 6, 'Observation': 98, 'Procedure': 107, 'Demographic': 40, 'Negation': 22, 'Qualifier': 206, 'Temporal': 85, 'Value': 139, 'Consent': 21}
False Negativies:
{'all': 1959, 'Condition': 538, 'Indication': 20, 'Drug': 182, 'Encounter': 13, 'Observation': 151, 'Procedure': 386, 'Demographic': 90, 'Negation': 24, 'Qualifier': 126, 'Temporal': 139, 'Value': 186, 'Consent': 104}
Overall Scores:


In [277]:
results_combo=results_combo.transpose()

In [279]:
results_combo

,0,1,2,3,4
entity,p,r,f,n_true,n_pred
all,0.827879,0.72486,0.772952,7096.0,6234.0
Condition,0.866495,0.757548,0.808367,1864.0,1940.0
Indication,0.731707,0.818182,0.772532,195.0,123.0
Drug,0.877682,0.692047,0.773888,489.0,466.0
Encounter,0.75,0.580645,0.654545,29.0,24.0
Observation,0.840131,0.773273,0.805317,701.0,613.0
Procedure,0.827697,0.571111,0.675871,666.0,621.0
Demographic,0.879154,0.76378,0.817416,359.0,331.0
Negation,0.832061,0.819549,0.825758,181.0,131.0
